# Establecer y obtener hiperparámetros en scikit-learn

Recordemos que los hiperparámetros se refieren a los parámetros que controlan el proceso
de aprendizaje de un modelo predictivo y son específicos para cada familia de modelos.
Además, el conjunto óptimo de hiperparámetros es específico para cada conjunto de datos y,
por lo tanto, siempre deben optimizarse.

Este notebook muestra cómo se puede obtener y establecer el valor de un hiperparámetro en un
estimador de scikit-learn.

No deben confundirse con los parámetros ajustados, resultantes del
entrenamiento. Estos parámetros ajustados son reconocibles en scikit-learn porque
se escriben con un guion bajo final `_`, por ejemplo `model.coef_`.

Comenzamos cargando el conjunto de datos del censo de adultos (adult census) y solo usamos las
características numéricas.

In [2]:
import pandas as pd

adult_census = pd.read_csv("adult_census.csv")

target_name = "class"
numerical_columns = ["age", "capital-gain", "capital-loss", "hours-per-week"]

target = adult_census[target_name]
data = adult_census[numerical_columns]

Nuestros datos son solo numéricos.

In [3]:
data.head()

,age,capital-gain,capital-loss,hours-per-week
0,25,0,0,40
1,38,0,0,50
2,28,0,0,40
3,44,7688,0,40
4,18,0,0,30


Vamos a crear un modelo predictivo simple, compuesto por un escalador (scaler) seguido de un clasificador de regresión logística.

Como se mencionó antes, muchos modelos, incluidos los lineales, funcionan
mejor si todas las características tienen una escala similar. Para este propósito, usamos un `StandardScaler`, que transforma los datos reescalando las características.

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

model = Pipeline(
    steps=[
        ("preprocessor", StandardScaler()),
        ("classifier", LogisticRegression()),
    ]
)

Podemos evaluar el rendimiento de generalización del modelo mediante
validación cruzada.

In [5]:
from sklearn.model_selection import cross_validate

cv_results = cross_validate(model, data, target)
scores = cv_results["test_score"]
print(
    "Puntuación de precisión (accuracy) mediante validación cruzada:\n"
    f"{scores.mean():.3f} ± {scores.std():.3f}"
)

Puntuación de precisión (accuracy) mediante validación cruzada:
0.800 ± 0.003


Creamos un modelo con el valor `C` predeterminado, que es igual a 1. Si quisiéramos
usar un hiperparámetro `C` diferente, podríamos haberlo hecho cuando creamos el
objeto `LogisticRegression` con algo como `LogisticRegression(C=1e-3)`.

Nota:
Para obtener más información sobre el hiperparámetro del modelo, consulta la [documentación](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).

También podemos cambiar el hiperparámetro de un modelo después de que haya sido creado
con el método `set_params`, que está disponible para todos los
estimadores de scikit-learn. Por ejemplo, podemos establecer `C=1e-3`, ajustar y evaluar el modelo:

In [6]:
model.set_params(classifier__C=1e-3)
cv_results = cross_validate(model, data, target)
scores = cv_results["test_score"]
print(
    "Puntuación de precisión (accuracy) mediante validación cruzada:\n"
    f"{scores.mean():.3f} ± {scores.std():.3f}"
)

Puntuación de precisión (accuracy) mediante validación cruzada:
0.787 ± 0.002


Cuando el modelo de interés es un `Pipeline`, los nombres de los hiperparámetros tienen
la forma `<nombre_modelo>__<nombre_hiperparametro>` (observa el doble guion bajo en
el medio). En nuestro caso, `classifier` proviene de la definición del `Pipeline`
y `C` es el nombre del hiperparámetro de `LogisticRegression`.

En general, puedes usar el método `get_params` en los modelos de scikit-learn para listar
todos los hiperparámetros con sus valores. Por ejemplo, si quieres obtener todos
los nombres de los hiperparámetros, puedes usar:

In [ ]:
for parameter in model.get_params():
    print(parameter)

memory
steps
transform_input
verbose
preprocessor
classifier
preprocessor__copy
preprocessor__with_mean
preprocessor__with_std
classifier__C
classifier__class_weight
classifier__dual
classifier__fit_intercept
classifier__intercept_scaling
classifier__l1_ratio
classifier__max_iter
classifier__multi_class
classifier__n_jobs
classifier__penalty
classifier__random_state
classifier__solver
classifier__tol
classifier__verbose
classifier__warm_start


`.get_params()` devuelve un `dict` cuyas claves son los nombres de los hiperparámetros y
cuyos valores son los valores de los hiperparámetros. Si quieres obtener el valor de un
solo hiperparámetro, por ejemplo `classifier__C`, puedes usar:

In [7]:
model.get_params()["classifier__C"]

0.001

Podemos variar sistemáticamente el valor de C para ver si hay un valor óptimo.

In [8]:
for C in [1e-3, 1e-2, 1e-1, 1, 10]:
    model.set_params(classifier__C=C)
    cv_results = cross_validate(model, data, target)
    scores = cv_results["test_score"]
    print(
        f"Puntuación de precisión (accuracy) mediante validación cruzada con C={C}:\n"
        f"{scores.mean():.3f} ± {scores.std():.3f}"
    )

Puntuación de precisión (accuracy) mediante validación cruzada con C=0.001:
0.787 ± 0.002
Puntuación de precisión (accuracy) mediante validación cruzada con C=0.01:
0.799 ± 0.003
Puntuación de precisión (accuracy) mediante validación cruzada con C=0.1:
0.800 ± 0.003
Puntuación de precisión (accuracy) mediante validación cruzada con C=1:
0.800 ± 0.003
Puntuación de precisión (accuracy) mediante validación cruzada con C=10:
0.800 ± 0.003


Podemos ver que mientras C sea lo suficientemente alto, el modelo parece funcionar bien.

Lo que hicimos aquí es muy manual: implica explorar los valores de C y
elegir el mejor manualmente. En la próxima sesión, veremos cómo hacer esto
automáticamente.

<div class="admonition warning alert alert-danger">
<p class="first admonition-title" style="font-weight: bold;">Advertencia</p>
<p class="last">Cuando evaluamos una familia de modelos en datos de prueba y elegimos el de mejor rendimiento,
no podemos confiar en la precisión de predicción correspondiente, y necesitamos aplicar
el modelo seleccionado a datos nuevos. De hecho, los datos de prueba se han utilizado para seleccionar
el modelo y, por lo tanto, ya no son independientes de este modelo.</p>
</div>

En este notebook hemos visto:

- cómo usar `get_params` y `set_params` para obtener los hiperparámetros de un modelo
  y establecerlos.